### Load model created by analyse_docs(...)

In [7]:
import os.path
import fasttext
import pickle as pkl
import shutup

from tqdm import tqdm
from huggingface_hub import hf_hub_download
from bertopic import BERTopic
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

shutup.please()

base_path = "/home/s371513/ernie/"
out_path = "/data/konsti_data/out/noto/"

topic_model = BERTopic.load(os.path.join(base_path, "model"))
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)

tmodel = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")



### Label Topics

In [2]:
custom_topics = {}

for i in range(len(topic_model.get_topic_info())):
    s = ""
    
    for tup in topic_model.get_topic(i):    
        try:
            impact_factor = float(tup[1])
            topic_keyword = tup[0]
        except Exception as _:
            impact_factor = float(tup[0])
            topic_keyword = tup[1]
            
        encoded_hi = tokenizer(topic_keyword, return_tensors="pt")
        generated_tokens = tmodel.generate(**encoded_hi)
        token = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        s += str(token) + " [" + str(impact_factor) + "]\n"
    
    topic = input("----------------------------\nTopics already created: \n{}\n----------Topic {}----------\n{}----------------------------".format(set(custom_topics.values()), str(i), s))
        
    custom_topics[i] = topic

with open(os.path.join(base_path, "custom_topics.pkl"), "wb+") as f:
    pkl.dump(custom_topics, f)